In [5]:
#imports
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import glob
import tensorflow as tf
import datetime

random.seed(108)

In [6]:
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

d:\CursPaper\yolov7


Cloning into 'yolov7'...


  Using cached protobuf-4.21.2-cp310-abi3-win_amd64.whl (524 kB)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.22.4
    Uninstalling protobuf-4.22.4:
      Successfully uninstalled protobuf-4.22.4


ERROR: Could not install packages due to an OSError: [WinError 5] Отказано в доступе: 'C:\\Users\\denzi\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\google\\~-pb\\_message.pyd'
Check the permissions.


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\denzi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="mRs9vg8hQcE37xjAudq2")
project = rf.workspace("saratov-state-uni").project("number-detection-s85cr")
dataset = project.version(3).download("yolov7")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 4.21.2 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\denzi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     -------------------------------------- 56.3/56.3 kB 582.9 kB/s eta 0:00:00
     -------------------------------------- 178.7/178.7 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 58.8/58.8 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 67.8/67.8 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 54.5/54.5 kB 2.8 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9681 sha256=cac04d8f6c4fda67babf3757700de38de0abd0311f84e9f5b32be20055b49991
  Stored in directory: c:\users\denzi\appdata\local\packages\pythonsoftwarefoundation.python.3.11_qbz5n2kfra8p0\localcache\local\pip\cache\wheels\f0\e4\9c\b3ed593784fe0147db216173b4046fce93fae7130b29e4464e
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Su

Extracting Dataset Version Zip to number-detection-3 in yolov7pytorch:: 100%|██████████| 1388/1388 [00:00<00:00, 2556.17it/s]


In [8]:
!wandb login e478cc6c0a4343a9912bfe3a45254bbf4c6139c1

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\denzi/.netrc


In [10]:
%%writefile cfg/training/yolov7_bus.yaml
# parameters
nc: 1  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple
 
# anchors
anchors:
  - [12,16, 19,36, 40,28]  # P3/8
  - [36,75, 76,55, 72,146]  # P4/16
  - [142,110, 192,243, 459,401]  # P5/32
 
# yolov7 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [32, 3, 1]],  # 0
  
  [-1, 1, Conv, [64, 3, 2]],  # 1-P1/2     
  [-1, 1, Conv, [64, 3, 1]],
  
  [-1, 1, Conv, [128, 3, 2]],  # 3-P2/4 
  [-1, 1, Conv, [64, 1, 1]],
  [-2, 1, Conv, [64, 1, 1]],
  [-1, 1, Conv, [64, 3, 1]],
  [-1, 1, Conv, [64, 3, 1]],
  [-1, 1, Conv, [64, 3, 1]],
  [-1, 1, Conv, [64, 3, 1]],
  [[-1, -3, -5, -6], 1, Concat, [1]],
  [-1, 1, Conv, [256, 1, 1]],  # 11
        
  [-1, 1, MP, []],
  [-1, 1, Conv, [128, 1, 1]],
  [-3, 1, Conv, [128, 1, 1]],
  [-1, 1, Conv, [128, 3, 2]],
  [[-1, -3], 1, Concat, [1]],  # 16-P3/8 
  [-1, 1, Conv, [128, 1, 1]],
  [-2, 1, Conv, [128, 1, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [[-1, -3, -5, -6], 1, Concat, [1]],
  [-1, 1, Conv, [512, 1, 1]],  # 24
        
  [-1, 1, MP, []],
  [-1, 1, Conv, [256, 1, 1]],
  [-3, 1, Conv, [256, 1, 1]],
  [-1, 1, Conv, [256, 3, 2]],
  [[-1, -3], 1, Concat, [1]],  # 29-P4/16 
  [-1, 1, Conv, [256, 1, 1]],
  [-2, 1, Conv, [256, 1, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [[-1, -3, -5, -6], 1, Concat, [1]],
  [-1, 1, Conv, [1024, 1, 1]],  # 37
        
  [-1, 1, MP, []],
  [-1, 1, Conv, [512, 1, 1]],
  [-3, 1, Conv, [512, 1, 1]],
  [-1, 1, Conv, [512, 3, 2]],
  [[-1, -3], 1, Concat, [1]],  # 42-P5/32 
  [-1, 1, Conv, [256, 1, 1]],
  [-2, 1, Conv, [256, 1, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [[-1, -3, -5, -6], 1, Concat, [1]],
  [-1, 1, Conv, [1024, 1, 1]],  # 50
  ]
 
# yolov7 head
head:
  [[-1, 1, SPPCSPC, [512]], # 51
  
  [-1, 1, Conv, [256, 1, 1]],
  [-1, 1, nn.Upsample, [None, 2, 'nearest']],
  [37, 1, Conv, [256, 1, 1]], # route backbone P4
  [[-1, -2], 1, Concat, [1]],
  
  [-1, 1, Conv, [256, 1, 1]],
  [-2, 1, Conv, [256, 1, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [[-1, -2, -3, -4, -5, -6], 1, Concat, [1]],
  [-1, 1, Conv, [256, 1, 1]], # 63
  
  [-1, 1, Conv, [128, 1, 1]],
  [-1, 1, nn.Upsample, [None, 2, 'nearest']],
  [24, 1, Conv, [128, 1, 1]], # route backbone P3
  [[-1, -2], 1, Concat, [1]],
  
  [-1, 1, Conv, [128, 1, 1]],
  [-2, 1, Conv, [128, 1, 1]],
  [-1, 1, Conv, [64, 3, 1]],
  [-1, 1, Conv, [64, 3, 1]],
  [-1, 1, Conv, [64, 3, 1]],
  [-1, 1, Conv, [64, 3, 1]],
  [[-1, -2, -3, -4, -5, -6], 1, Concat, [1]],
  [-1, 1, Conv, [128, 1, 1]], # 75
      
  [-1, 1, MP, []],
  [-1, 1, Conv, [128, 1, 1]],
  [-3, 1, Conv, [128, 1, 1]],
  [-1, 1, Conv, [128, 3, 2]],
  [[-1, -3, 63], 1, Concat, [1]],
  
  [-1, 1, Conv, [256, 1, 1]],
  [-2, 1, Conv, [256, 1, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [-1, 1, Conv, [128, 3, 1]],
  [[-1, -2, -3, -4, -5, -6], 1, Concat, [1]],
  [-1, 1, Conv, [256, 1, 1]], # 88
      
  [-1, 1, MP, []],
  [-1, 1, Conv, [256, 1, 1]],
  [-3, 1, Conv, [256, 1, 1]],
  [-1, 1, Conv, [256, 3, 2]],
  [[-1, -3, 51], 1, Concat, [1]],
  
  [-1, 1, Conv, [512, 1, 1]],
  [-2, 1, Conv, [512, 1, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [-1, 1, Conv, [256, 3, 1]],
  [[-1, -2, -3, -4, -5, -6], 1, Concat, [1]],
  [-1, 1, Conv, [512, 1, 1]], # 101
  
  [75, 1,   , [256, 3, 1]],
  [88, 1, RepConv, [512, 3, 1]],
  [101, 1, RepConv, [1024, 3, 1]],
 
  [[102,103,104], 1, IDetect, [nc, anchors]],   # Detect(P3, P4, P5)
  ]

Writing cfg/training/yolov7_bus.yaml


In [ ]:
import pytorch
!pip show torch

In [4]:
%cd yolov7/
!python train.py --epochs 100 --batch-size 8 --data number-detection-3/data.yaml \
--img 640 640 --cfg cfg/training/yolov7_bus.yaml --weights 'yolov7_training.pt' \
--name yolov7_bus_res --hyp data/hyp.scratch.custom.yaml

[WinError 2] Не удается найти указанный файл: 'yolov7/'
d:\CursPaper\yolov7


Traceback (most recent call last):
  File "d:\CursPaper\yolov7\train.py", line 12, in <module>
    import torch.distributed as dist
  File "C:\Users\denzi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\__init__.py", line 122, in <module>
    raise err
OSError: [WinError 1455] Файл подкачки слишком мал для завершения операции. Error loading "C:\Users\denzi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\lib\cufft64_10.dll" or one of its dependencies.
